In [2]:
import pandas as pd
import json

In [3]:
with open('datasets/mmlu_ro/ro_test.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

df = pd.DataFrame(data)


In [4]:
df.head()

,instruction,option_a,option_b,option_c,option_d,answer,id
0,Pentru care dintre cele două scenarii face per...,"Greșit, Greșit","Greșit, Nu greșit","Nu greșit, Greșit","Nu greșit, Nu greșit",B,moral_scenarios/test/106
1,Grupul Națiunii Islamice a apelat la:,Imigranți din a doua generație originari din s...,Americani albi care au vrut să se convertească...,Afro-americani care s-au simțit excluși din «t...,Afro-caraibieni care trăiau în zonele centrale...,C,sociology/test/49
2,Care parte a spectrului electromagnetic are ce...,Razele gamma,Raze X,Undele radio,Undele microundelor,A,miscellaneous/test/535
3,Scopul principal al declarației de activitate ...,Furnizorilor de resurse.,Managerilor.,Beneficiarilor.,Organelor de reglementare a statului.,A,professional_accounting/test/251
4,O femeie era proprietara unei ferme in proprie...,"femeia era proprietarul legitim si, prin urmar...","femeia avea dreptul la posesie, deoarece poses...","barbatul avea dreptul la posesie, deoarece pos...","barbatul avea dreptul la posesie, datorita pri...",D,professional_law/test/879


In [5]:
llama_3_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Sunteți un asistent util, respectuos și onest. Veți primi o instrucțiune și 4 variante de răspuns. O singură variantă este corectă. Răspundeți doar cu litera corespunzătoare variantei corecte.<|eot_id|><|start_header_id|>user<|end_header_id|>

{instruction}
A. {option_a}
B. {option_b}
C. {option_c}
D. {option_d}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

"""

llama_2_prompt = """<s>[INST] <<SYS>>
Sunteți un asistent util, respectuos și onest. Veți primi o instrucțiune și 4 variante de răspuns. O singură variantă este corectă. Răspundeți doar cu litera corespunzătoare variantei corecte.
<</SYS>>

{instruction}
A. {option_a}
B. {option_b}
C. {option_c}
D. {option_d} [/INST]
"""

alpaca_prompt = """Mai jos este prezentată o instrucțiune care descrie o sarcină, însoțită de o intrare care oferă un context suplimentar. Scrieți un răspuns care să completeze în mod corespunzător cererea. Trebuie să răspundeți numai în limba română.

### Instrucțiune:
{instruction}

### Intrare:
A. {option_a}
B. {option_b}
C. {option_c}
D. {option_d}

### Răspuns:
"""

In [28]:
unique_ids = df['id'].unique()
print(unique_ids)

['econometrics/dev/1' 'professional_medicine/dev/0' 'philosophy/dev/1'
 'high_school_macroeconomics/dev/0' 'public_relations/dev/2'
 'econometrics/dev/2' 'logical_fallacies/dev/4'
 'high_school_world_history/dev/2' 'world_religions/dev/3'
 'security_studies/dev/1' 'clinical_knowledge/dev/0'
 'high_school_microeconomics/dev/2' 'college_biology/dev/1'
 'human_aging/dev/2' 'miscellaneous/dev/2' 'high_school_chemistry/dev/4'
 'computer_security/dev/3' 'medical_genetics/dev/1' 'sociology/dev/3'
 'virology/dev/4' 'high_school_european_history/dev/1'
 'conceptual_physics/dev/0' 'abstract_algebra/dev/4'
 'business_ethics/dev/4' 'marketing/dev/2' 'high_school_mathematics/dev/3'
 'high_school_statistics/dev/4' 'computer_security/dev/1'
 'jurisprudence/dev/4' 'anatomy/dev/1' 'high_school_mathematics/dev/2'
 'professional_law/dev/0' 'human_aging/dev/1'
 'high_school_mathematics/dev/0' 'professional_accounting/dev/1'
 'high_school_chemistry/dev/2' 'us_foreign_policy/dev/2'
 'professional_law/dev/4'

In [29]:
# Split the "id" column by '/'
categories = df['id'].str.split('/')

# Extract the category before '/'
categories_before_slash = categories.str[0]

# Get unique categories
unique_categories = categories_before_slash.unique()
print(unique_categories)


['econometrics' 'professional_medicine' 'philosophy'
 'high_school_macroeconomics' 'public_relations' 'logical_fallacies'
 'high_school_world_history' 'world_religions' 'security_studies'
 'clinical_knowledge' 'high_school_microeconomics' 'college_biology'
 'human_aging' 'miscellaneous' 'high_school_chemistry' 'computer_security'
 'medical_genetics' 'sociology' 'virology' 'high_school_european_history'
 'conceptual_physics' 'abstract_algebra' 'business_ethics' 'marketing'
 'high_school_mathematics' 'high_school_statistics' 'jurisprudence'
 'anatomy' 'professional_law' 'professional_accounting'
 'us_foreign_policy' 'high_school_geography' 'high_school_physics'
 'college_physics' 'college_chemistry' 'college_computer_science'
 'prehistory' 'machine_learning' 'high_school_psychology'
 'international_law' 'astronomy' 'global_facts' 'moral_scenarios'
 'professional_psychology' 'elementary_mathematics'
 'electrical_engineering' 'formal_logic' 'college_mathematics'
 'high_school_us_history' '

In [6]:
import requests

def generate_response(prompt):
    url = "http://10.198.110.23:8000/generate"

    payload = {
        "prompt": prompt,
        "n": 1,
        "temperature": 0.95,
        "max_tokens": 1024
    }

    response = requests.post(url, json=payload)

    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code)
        return None

def get_chosen_answer(row):
    response_data = generate_response(row['formatted_prompt'])
    if response_data:
        answer = response_data['text'][0].split('\n')[-1]
        return answer
    else:
        return None


In [7]:
pd.set_option('display.max_colwidth', None)

# Increase maximum number of rows displayed
pd.set_option('display.max_rows', None)

In [8]:
df['formatted_prompt'] = df.apply(lambda row: llama_3_prompt.format(**row).strip(), axis=1)

df['formatted_prompt'][1]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nSunteți un asistent util, respectuos și onest. Veți primi o instrucțiune și 4 variante de răspuns. O singură variantă este corectă. Răspundeți doar cu litera corespunzătoare variantei corecte.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGrupul Națiunii Islamice a apelat la:\nA. Imigranți din a doua generație originari din subcontinentul asiatic născuți în Marea Britanie\nB. Americani albi care au vrut să se convertească la islam\nC. Afro-americani care s-au simțit excluși din «toporul etnic» din SUA\nD. Afro-caraibieni care trăiau în zonele centrale ale orașelor și aveau o cultură distinctivă a tinerilor<|eot_id|><|start_header_id|>assistant<|end_header_id|>'

In [9]:
formatted_prompt = df['formatted_prompt'][1]

formatted_prompt

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nSunteți un asistent util, respectuos și onest. Veți primi o instrucțiune și 4 variante de răspuns. O singură variantă este corectă. Răspundeți doar cu litera corespunzătoare variantei corecte.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGrupul Națiunii Islamice a apelat la:\nA. Imigranți din a doua generație originari din subcontinentul asiatic născuți în Marea Britanie\nB. Americani albi care au vrut să se convertească la islam\nC. Afro-americani care s-au simțit excluși din «toporul etnic» din SUA\nD. Afro-caraibieni care trăiau în zonele centrale ale orașelor și aveau o cultură distinctivă a tinerilor<|eot_id|><|start_header_id|>assistant<|end_header_id|>'

In [10]:
df['chosen_answer'] = df.apply(lambda row: get_chosen_answer(row), axis=1)


In [13]:
df['chosen_answer'].head()

0                                                                                                                   B. Greșit, Nu greșit
1                                                                                                                                    (c)
2                                                                                                      A (Pentru lungimi de undă scurte)
3                                                                                                                                      C
4    B. Femeia avea dreptul la posesie, deoarece posesia intermitentă a bărbațului a fost insuficientă pentru a-l face un posesor ostil.
Name: chosen_answer, dtype: object

In [14]:
new_df = df[['instruction', 'option_a', 'option_b', 'option_c', 'option_d', 'answer', 'chosen_answer', 'id']].copy()
new_df.rename(columns={'answer': 'correct_answer'}, inplace=True)

In [15]:
new_df.head()

,instruction,option_a,option_b,option_c,option_d,correct_answer,chosen_answer,id
0,"Pentru care dintre cele două scenarii face personajul principal (care folosește I/me/my) ceva clar imoral, conform standardelor morale obișnuite în SUA în 2020? Scenariul 1 | Am închiriat un băiat elegant și sexy pentru după amiază. Scenariul 2 | Am relocat refugiații în case.","Greșit, Greșit","Greșit, Nu greșit","Nu greșit, Greșit","Nu greșit, Nu greșit",B,"B. Greșit, Nu greșit",moral_scenarios/test/106
1,Grupul Națiunii Islamice a apelat la:,Imigranți din a doua generație originari din subcontinentul asiatic născuți în Marea Britanie,Americani albi care au vrut să se convertească la islam,Afro-americani care s-au simțit excluși din «toporul etnic» din SUA,Afro-caraibieni care trăiau în zonele centrale ale orașelor și aveau o cultură distinctivă a tinerilor,C,(c),sociology/test/49
2,Care parte a spectrului electromagnetic are cea mai scurtă lungime de undă?,Razele gamma,Raze X,Undele radio,Undele microundelor,A,A (Pentru lungimi de undă scurte),miscellaneous/test/535
3,Scopul principal al declarației de activitate a unei organizații non-profit este de a furniza informații relevante:,Furnizorilor de resurse.,Managerilor.,Beneficiarilor.,Organelor de reglementare a statului.,A,C,professional_accounting/test/251
4,"O femeie era proprietara unei ferme in proprietate simpla. Femeia a parasit ferma in 1986 pentru a se muta in oras. Un barbat a luat posesia terenului in 1987 ca posesor ostil si a ramas pe teren intre 1987 si 1997; apoi a parasit proprietatea pentru a-si ingriji mama bolnava, timp de un an, revenind in anul urmator, 1999. Nepotul barbatului a acceptat sa locuiasca pe ferma si sa aiba grija de ea in timpul absentei unchiului sau. Cand barbatul s-a intors din casa mamei sale in 1999, a continuat sa fie in posesie pana in 2010, cand femeia a intentat o actiune de evacuare. Se presupune ca perioada legala pentru posedarea ostila in acest stat este de 20 de ani. Cel mai probabil, instanta ar considera ca","femeia era proprietarul legitim si, prin urmare, avea dreptul sa posede ferma.","femeia avea dreptul la posesie, deoarece posesia intermitenta a barbatului a fost insuficienta pentru a-l face un posesor ostil.","barbatul avea dreptul la posesie, deoarece posesia sa a fost deschisa, notorie, continua si ostila, asa cum este necesar pentru perioada legala.","barbatul avea dreptul la posesie, datorita privity dintre barbat si nepotul sau, care l-a ajutat sa indeplineasca perioada legala necesara.",D,"B. Femeia avea dreptul la posesie, deoarece posesia intermitentă a bărbațului a fost insuficientă pentru a-l face un posesor ostil.",professional_law/test/879


In [16]:
new_df.to_csv('mmlu_llama3_instruct.csv', index=False, encoding='utf-8')